In [1]:
%matplotlib notebook
import cvxpy as cp
import dccp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.utils import shuffle
import matplotlib.patches as mpatches
import json
import random
import math
import os, psutil
from datetime import datetime

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
np.random.seed(0)

TRAIN_SLOPE = 2
EVAL_SLOPE = 5
X_LOWER_BOUND = -10
X_UPPER_BOUND = 10
SEED = 2

# Utils

In [2]:
def split_data(X, Y, percentage):
    num_val = int(len(X)*percentage)
    return X[num_val:], Y[num_val:], X[:num_val], Y[:num_val]

def shuffle(X, Y):
    torch.manual_seed(0)
    np.random.seed(0)
    data = torch.cat((Y, X), 1)
    data = data[torch.randperm(data.size()[0])]
    X = data[:, 1:]
    Y = data[:, 0]
    return X, Y

def conf_mat(Y1, Y2):
    num_of_samples = len(Y1)
    mat = confusion_matrix(Y1, Y2, labels=[-1, 1])*100/num_of_samples
    acc = np.trace(mat)
    return mat, acc

def calc_accuracy(Y, Ypred):
    num = len(Y)
    temp = Y - Ypred
    acc = len(temp[temp == 0])*1./num
    return acc

# CCP classes

In [11]:
class CCP:
    def __init__(self, x_dim, batch_size, funcs, scale):
        self.f_derivative = funcs["f_derivative"]
        self.g = funcs["g"]
        self.c = funcs["c"]
        self.batch_size = batch_size
        
        self.x = cp.Variable((batch_size, x_dim))
        self.xt = cp.Parameter((batch_size, x_dim))
        self.r = cp.Parameter((batch_size, x_dim))
        self.w = cp.Parameter(x_dim)
        self.b = cp.Parameter(1)
        self.slope = cp.Parameter(1)

        target = cp.diag(self.x@(self.f_derivative(self.xt, self.w, self.b, self.slope).T))-self.g(self.x, self.w, self.b, self.slope)-self.c(self.x, self.r, x_dim, scale)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        self.prob = cp.Problem(cp.Maximize(cp.sum(target)), constraints)
        
    def ccp(self, r, num_iterations):
        """
        numpy to numpy
        """
        self.xt.value = r
        self.r.value = r
        result = self.prob.solve()
        diff = np.linalg.norm(self.xt.value - self.x.value)
        for i in range(num_iterations):
            self.xt.value = self.x.value
            result = self.prob.solve()
            diff = np.linalg.norm(self.x.value - self.xt.value)/self.batch_size
        return self.x.value
    
    def optimize_X(self, X, w, b, slope, num_iterations):
        """
        tensor to tensor
        """
        w = w.detach().numpy()
        b = b.detach().numpy()
        slope = np.full(1, slope)
        X = X.numpy()
        
        self.w.value = w
        self.b.value = b
        self.slope.value = slope
        return torch.from_numpy(self.ccp(X, num_iterations))
        # return torch.stack([torch.from_numpy(self.ccp(x)) for x in X])

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
class DELTA():
    
    def __init__(self, x_dim, funcs, scale):
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.b = cp.Parameter(1, value = np.random.randn(1))
        self.f_der = cp.Parameter(x_dim, value = np.random.randn(x_dim))

        target = self.x@self.f_der-self.g(self.x, self.w, self.b, TRAIN_SLOPE)-self.c(self.x, self.r, x_dim, scale)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        objective = cp.Maximize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.b, self.f_der],
                                variables=[self.x])
        
    def optimize_X(self, X, w, b, F_DER):
        return self.layer(X, w, b, F_DER)[0]

# Gain & Cost functions

In [13]:
def score(x, w, b):
    return x@w + b

def f(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) + 1)]), 2)

def g(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) - 1)]), 2)

def c(x, r, x_dim, scale):
    return (scale)*cp.sum_squares(x-r)

def f_derivative(x, w, b, slope):
    return 0.5*cp.multiply(slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1)), w)
    
def f_batch(x, w, b, slope):
    return 0.5*cp.norm(cp.vstack([np.ones(x.shape[0]), (slope*score(x, w, b) + 1)]), 2, axis=0)

def g_batch(x, w, b, slope):
    return 0.5*cp.norm(cp.vstack([np.ones((1, x.shape[0])), cp.reshape((slope*score(x, w, b) - 1), (1, x.shape[0]))]), 2, axis=0)

def c_batch(x, r, x_dim, scale):
    return (scale)*cp.square(cp.norm(x-r, 2, axis=1))

def f_derivative_batch(x, w, b, slope):
    nablas = 0.5*slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1))
    return cp.reshape(nablas, (nablas.shape[0], 1))@cp.reshape(w, (1, x.shape[1]))

# Model

In [17]:
class MyStrategicModel(torch.nn.Module):
    def __init__(self, x_dim, batch_size, funcs, funcs_batch, train_slope, eval_slope, scale, strategic=False):
        torch.manual_seed(0)
        np.random.seed(0)

        super(MyStrategicModel, self).__init__()
        self.x_dim = x_dim
        self.batch_size = batch_size
        self.train_slope, self.eval_slope = train_slope, eval_slope
        self.w = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(x_dim, dtype=torch.float64, requires_grad=True)))
        self.b = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(1, dtype=torch.float64, requires_grad=True)))
#         self.w = torch.nn.parameter.Parameter(torch.zeros(x_dim, dtype=torch.float64, requires_grad=True))
#         self.b = torch.nn.parameter.Parameter(torch.zeros(1, dtype=torch.float64, requires_grad=True))
        self.strategic = strategic
        self.ccp = CCP(x_dim, batch_size, funcs_batch, scale)
        self.delta = DELTA(x_dim, funcs, scale)
        self.ccp_time = 0
        self.total_time = 0

    def forward(self, X, num_iterations, evaluation=False):
        if self.strategic:
            if evaluation:
                t1 = time.time()
                XT = self.ccp.optimize_X(X, self.w, self.b, self.eval_slope, num_iterations)
                self.ccp_time += time.time()-t1
                X_opt = XT
            else:
                t1 = time.time()
                XT = self.ccp.optimize_X(X, self.w, self.b, self.train_slope, num_iterations)
                self.ccp_time += time.time()-t1
                F_DER = self.get_f_ders(XT, self.train_slope)
                X_opt = self.delta.optimize_X(X, self.w, self.b, F_DER) # Xopt should be equal to XT but we do it again for the gradients
            output = self.score(X_opt)
        else:
            output = self.score(X)        
        return output
    
    def optimize_X(self, X, evaluation=False):
        slope = self.eval_slope if evaluation else self.train_slope
        return self.ccp.optimize_X(X, self.w, self.b, slope)
    
    def normalize_weights(self):
        with torch.no_grad():
            norm = torch.sqrt(torch.sum(self.w**2) + self.b**2)
            self.w /= norm
            self.b /= norm

    def score(self, x):
        return x@self.w + self.b
    
    def get_f_ders(self, XT, slope):
        # return torch.stack([0.5*slope*((slope*self.score(xt) + 1)/torch.sqrt((slope*self.score(xt) + 1)**2 + 1))*self.w for xt in XT])
        nablas = 0.5*slope*((slope*self.score(XT) + 1)/torch.sqrt((slope*self.score(XT) + 1)**2 + 1))
        return torch.reshape(nablas, (len(nablas), 1))@torch.reshape(self.w, (1, len(self.w)))

    def calc_accuracy(self, Y, Y_pred):
        Y_pred = torch.sign(Y_pred)
        num = len(Y)
        temp = Y - Y_pred
        acc = len(temp[temp == 0])*1./num        
        return acc
    
    def evaluate(self, X, Y):      
        return self.calc_accuracy(Y, self.forward(X, evaluation=True))
    
    def loss(self, Y, Y_pred):
        return torch.mean(torch.clamp(1 - Y_pred * Y, min=0))
    
    def save_model(self, train_errors, val_errors, train_losses, val_losses, info, path, comment=None):
        if comment is not None:
            path += "/" + comment
            
        filename = path + "/model.pt"
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        torch.save(self.state_dict(), filename)
                
        pd.DataFrame(np.array(train_errors)).to_csv(path + '/train_errors.csv')
        pd.DataFrame(np.array(val_errors)).to_csv(path + '/val_errors.csv')
        pd.DataFrame(np.array(train_losses)).to_csv(path + '/train_losses.csv')
        pd.DataFrame(np.array(val_losses)).to_csv(path + '/val_losses.csv')
        
        with open(path + "/info.txt", "w") as f:
            f.write(info)
    
    def load_model(self, filename):
        self.load_state_dict(torch.load(filename))
        self.eval()
    
    def fit(self, path, X, Y, Xval, Yval, opt, opt_kwargs={"lr":1e-3}, batch_size=128, epochs=100, verbose=False, callback=None, comment=None):
        train_dset = TensorDataset(X, Y)
        train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        test_dset = TensorDataset(Xval, Yval)
        test_loader = DataLoader(test_dset, batch_size=batch_size, shuffle=True)
        
        opt = opt(self.parameters(), **opt_kwargs)

        train_losses = []
        val_losses = []
        train_errors = []
        val_errors = []
        
        best_val_error = 1
        consecutive_no_improvement = 0

        total_time = time.time()
        for epoch in range(epochs):
            t1 = time.time()
            batch = 1
            train_losses.append([])
            train_errors.append([])
            for Xbatch, Ybatch in train_loader:
#                 try:
                opt.zero_grad()
                Ybatch_pred = self.forward(Xbatch, epoch + 1)
                l = self.loss(Ybatch, Ybatch_pred)
                l.backward()
                opt.step()
                train_losses[-1].append(l.item())
                with torch.no_grad():
                    e = self.calc_accuracy(Ybatch, Ybatch_pred)
                    train_errors[-1].append(1-e)
                if verbose:
                    print("batch %03d / %03d | loss: %3.5f | err: %3.5f" %
                          (batch, len(train_loader), np.mean(train_losses[-1]), np.mean(train_errors[-1])))
                batch += 1
                if callback is not None:
                    callback()
#                 except:
#                     print("failed")
                
            with torch.no_grad():
                total_loss = 0
                total_error = 0
                batch = 0
                for Xbatch, Ybatch in test_loader:
#                     try:
                    Yval_pred = self.forward(Xbatch, epoch + 1, evaluation=True)
                    val_loss = self.loss(Ybatch, Yval_pred).item()
                    total_loss += val_loss
                    val_error = 1-self.calc_accuracy(Ybatch, Yval_pred)
                    total_error += val_error
                    batch += 1
#                     except:
#                         print("failed")
                        
                avg_loss = total_loss/batch
                avg_error = total_error/batch
                val_losses.append(avg_loss)
                val_errors.append(avg_error)
                if avg_error < best_val_error:
                        consecutive_no_improvement = 0
                        best_val_error = avg_error
                        info = "training time in seconds: {}\nepoch: {}\nbatch size: {}\ntrain slope: {}\neval slope: {}\nlearning rate: {}\nvalidation loss: {}\nvalidation error: {}\n".format(
                        time.time()-total_time, epoch, batch_size, self.train_slope, self.eval_slope, opt_kwargs["lr"], avg_loss, avg_error)
                        self.save_model(train_errors, val_errors, train_losses, val_losses, info, path, comment)
                        print("model saved!")

                else:
                    consecutive_no_improvement += 1
                    if consecutive_no_improvement >= 4:
                        break
                    
            t2 = time.time()
            if verbose:
                print("------------- epoch %03d / %03d | time: %03d sec | loss: %3.5f | err: %3.5f" % (epoch + 1, epochs, t2-t1, val_losses[-1], val_errors[-1]))
        
        self.total_time = time.time()-total_time
        print("training time: {} seconds".format(self.total_time)) 
        return train_errors, val_errors, train_losses, val_losses

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
def gen_sklearn_data(x_dim, N, informative_frac=1, shift_range=1, scale_range=1, noise_frac=0.01):
    torch.manual_seed(0)
    np.random.seed(0)
    n_informative = int(informative_frac*x_dim)
    n_redundant = x_dim - n_informative
    shift_arr = shift_range*np.random.randn(x_dim)
    scale_arr = scale_range*np.random.randn(x_dim)
    X, Y = make_classification(n_samples=N, n_features=x_dim, n_informative=n_informative, n_redundant=n_redundant,
                               flip_y=noise_frac, shift=shift_arr, scale=scale_arr, random_state=0)
    Y[Y == 0] = -1
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_spam_data():
    torch.manual_seed(0)
    np.random.seed(0)
    path = r"C:\Users\sagil\Desktop\nir_project\tip_spam_data\IS_journal_tip_spam.arff"
    data, meta = arff.loadarff(path)
    df = pd.DataFrame(data)
    most_disc = ['qTips_plc', 'rating_plc', 'qEmail_tip', 'qContacts_tip', 'qURL_tip', 'qPhone_tip', 'qNumeriChar_tip', 'sentistrength_tip', 'combined_tip', 'qWords_tip', 'followers_followees_gph', 'qunigram_avg_tip', 'qTips_usr', 'indeg_gph', 'qCapitalChar_tip', 'class1']
    df = df[most_disc]
    df["class1"].replace({b'spam': -1, b'notspam': 1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y = df['class1'].values
    X = df.drop('class1', axis = 1).values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_card_fraud_data():
    torch.manual_seed(0)
    np.random.seed(0)
    df = pd.read_csv('C:/Users/sagil/Desktop/nir_project/card_fraud_dataset/creditcard.csv')

    rob_scaler = RobustScaler()

    df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
    df.drop(['Time','Amount'], axis=1, inplace=True)
    scaled_amount = df['scaled_amount']
    df.drop(['scaled_amount'], axis=1, inplace=True)
    df.insert(0, 'scaled_amount', scaled_amount)

    df["Class"].replace({1: -1, 0: 1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    # amount of fraud classes 492 rows.
    fraud_df = df.loc[df['Class'] == -1]
    non_fraud_df = df.loc[df['Class'] == 1][:492]

    normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

    # Shuffle dataframe rows
    df = normal_distributed_df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y = df['Class'].values
    X = df.drop('Class', axis = 1).values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_credit_default_data():
    torch.manual_seed(0)
    np.random.seed(0)
    url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
    df = pd.read_csv(url)
    df["NoDefaultNextMonth"].replace({0: -1}, inplace=True)
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

    df = df.drop(['Married', 'Single', 'Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60'], axis = 1)

    fraud_df = df.loc[df["NoDefaultNextMonth"] == -1]
    non_fraud_df = df.loc[df["NoDefaultNextMonth"] == 1][:6636]

    normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

    # Shuffle dataframe rows
    df = normal_distributed_df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    
    scaler = StandardScaler()
    df.loc[:, df.columns != "NoDefaultNextMonth"] = scaler.fit_transform(df.drop("NoDefaultNextMonth", axis=1)) 
    Y, X = df.iloc[:, 0].values, df.iloc[:, 1:].values
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

def load_financial_distress_data():
    torch.manual_seed(0)
    np.random.seed(0)
    data = pd.read_csv("C:/Users/sagil/Desktop/nir_project/financial_distress_data/Financial Distress.csv")

    data = data[data.columns.drop(list(data.filter(regex='x80')))] # Since it is a categorical feature with 37 features.
    x_dim = len(data.columns) - 3
    data.drop(['Time'], axis=1, inplace=True)

    data_grouped = data.groupby(['Company']).last()

    scaler = StandardScaler()
    data_grouped.loc[:, data_grouped.columns != "Financial Distress"] = scaler.fit_transform(data_grouped.drop("Financial Distress", axis=1))

    # Shuffle dataframe rows
    data_grouped = data_grouped.sample(frac=1, random_state=SEED).reset_index(drop=True)

    Y, X = data_grouped.iloc[:, 0].values, data_grouped.iloc[:, 1:].values
    for y in range(0,len(Y)): # Coverting target variable from continuous to binary form
        if Y[y] < -0.5:
              Y[y] = -1
        else:
              Y[y] = 1
    x_dim = len(X[0])
    X -= np.mean(X, axis=0)
    X /= np.std(X, axis=0)
    X /= math.sqrt(x_dim)
    return torch.from_numpy(X), torch.from_numpy(Y)

In [19]:
path = "C:/Users/sagil/Desktop/nir_project/models/runtime_varying_batch_size_dim5_inc_tol"
epochs = 5
x_dim = 5
scale = 1
X, Y = gen_sklearn_data(x_dim, 1024)
X, Y, Xval, Yval = split_data(X, Y, 0.25)
print(Xval.size())
print("percent of positive samples: {}%".format(100 * len(Y[Y == 1]) / len(Y)))

funcs = {"f": f, "g": g, "f_derivative": f_derivative, "c": c, "score": score}
funcs_batch = {"f": f_batch, "g": g_batch, "f_derivative": f_derivative_batch, "c": c_batch, "score": score}

total = []
ccp = []
for batch_size in (2**np.arange(9)).tolist():
    strategic_model = MyStrategicModel(x_dim, batch_size, funcs, funcs_batch, TRAIN_SLOPE, EVAL_SLOPE, scale=scale, strategic=True)
    strategic_model.fit(path, X, Y, Xval, Yval,
                        opt=torch.optim.Adam, opt_kwargs={"lr": (1e-1)},
                        batch_size=batch_size, epochs=epochs, verbose=True,
                       comment="batched")
    
    total_time = strategic_model.total_time
    ccp_time = strategic_model.ccp_time
    total.append(total_time)
    ccp.append(ccp_time)
    pd.DataFrame(np.array(total)).to_csv(path + '/total_timing_results.csv')
    pd.DataFrame(np.array(ccp)).to_csv(path + '/ccp_timing_results.csv')

torch.Size([256, 5])
percent of positive samples: 50.78125%
batch 001 / 768 | loss: 0.30994 | err: 0.00000
batch 002 / 768 | loss: 0.67513 | err: 0.50000


C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\l

batch 003 / 768 | loss: 0.45008 | err: 0.33333
batch 004 / 768 | loss: 0.82082 | err: 0.50000
batch 005 / 768 | loss: 0.65666 | err: 0.40000
batch 006 / 768 | loss: 0.83206 | err: 0.50000
batch 007 / 768 | loss: 0.71320 | err: 0.42857
batch 008 / 768 | loss: 0.83747 | err: 0.50000
batch 009 / 768 | loss: 0.74442 | err: 0.44444
batch 010 / 768 | loss: 0.66998 | err: 0.40000
batch 011 / 768 | loss: 0.64063 | err: 0.36364
batch 012 / 768 | loss: 0.62015 | err: 0.33333
batch 013 / 768 | loss: 0.57244 | err: 0.30769
batch 014 / 768 | loss: 0.59444 | err: 0.28571
batch 015 / 768 | loss: 0.59189 | err: 0.26667
batch 016 / 768 | loss: 0.55490 | err: 0.25000
batch 017 / 768 | loss: 0.53747 | err: 0.23529
batch 018 / 768 | loss: 0.52719 | err: 0.22222
batch 019 / 768 | loss: 0.58482 | err: 0.26316
batch 020 / 768 | loss: 0.55557 | err: 0.25000
batch 021 / 768 | loss: 0.68482 | err: 0.28571
batch 022 / 768 | loss: 0.82438 | err: 0.31818
batch 023 / 768 | loss: 0.78854 | err: 0.30435
batch 024 / 7

batch 178 / 768 | loss: 0.46297 | err: 0.17978
batch 179 / 768 | loss: 0.46038 | err: 0.17877
batch 180 / 768 | loss: 0.45783 | err: 0.17778
batch 181 / 768 | loss: 0.45530 | err: 0.17680
batch 182 / 768 | loss: 0.45279 | err: 0.17582
batch 183 / 768 | loss: 0.45032 | err: 0.17486
batch 184 / 768 | loss: 0.44787 | err: 0.17391
batch 185 / 768 | loss: 0.44545 | err: 0.17297
batch 186 / 768 | loss: 0.44306 | err: 0.17204
batch 187 / 768 | loss: 0.44069 | err: 0.17112
batch 188 / 768 | loss: 0.43834 | err: 0.17021
batch 189 / 768 | loss: 0.43602 | err: 0.16931
batch 190 / 768 | loss: 0.43373 | err: 0.16842
batch 191 / 768 | loss: 0.43146 | err: 0.16754
batch 192 / 768 | loss: 0.42921 | err: 0.16667
batch 193 / 768 | loss: 0.42699 | err: 0.16580
batch 194 / 768 | loss: 0.43489 | err: 0.17010
batch 195 / 768 | loss: 0.44680 | err: 0.17436
batch 196 / 768 | loss: 0.44453 | err: 0.17347
batch 197 / 768 | loss: 0.44227 | err: 0.17259
batch 198 / 768 | loss: 0.46088 | err: 0.17677
batch 199 / 7

batch 354 / 768 | loss: 0.47683 | err: 0.18644
batch 355 / 768 | loss: 0.47548 | err: 0.18592
batch 356 / 768 | loss: 0.47415 | err: 0.18539
batch 357 / 768 | loss: 0.47282 | err: 0.18487
batch 358 / 768 | loss: 0.47150 | err: 0.18436
batch 359 / 768 | loss: 0.47018 | err: 0.18384
batch 360 / 768 | loss: 0.46899 | err: 0.18333
batch 361 / 768 | loss: 0.47267 | err: 0.18560
batch 362 / 768 | loss: 0.47137 | err: 0.18508
batch 363 / 768 | loss: 0.47007 | err: 0.18457
batch 364 / 768 | loss: 0.46878 | err: 0.18407
batch 365 / 768 | loss: 0.46828 | err: 0.18356
batch 366 / 768 | loss: 0.46700 | err: 0.18306
batch 367 / 768 | loss: 0.46573 | err: 0.18256
batch 368 / 768 | loss: 0.46446 | err: 0.18207
batch 369 / 768 | loss: 0.47282 | err: 0.18428
batch 370 / 768 | loss: 0.47155 | err: 0.18378
batch 371 / 768 | loss: 0.47028 | err: 0.18329
batch 372 / 768 | loss: 0.46901 | err: 0.18280
batch 373 / 768 | loss: 0.46775 | err: 0.18231
batch 374 / 768 | loss: 0.47154 | err: 0.18449
batch 375 / 7

...

batch 056 / 096 | loss: 0.34860 | err: 0.14955
batch 057 / 096 | loss: 0.34348 | err: 0.14693
batch 058 / 096 | loss: 0.34592 | err: 0.14871
batch 059 / 096 | loss: 0.34529 | err: 0.14831
batch 060 / 096 | loss: 0.34338 | err: 0.14792
batch 061 / 096 | loss: 0.33868 | err: 0.14549
batch 062 / 096 | loss: 0.33404 | err: 0.14315
batch 063 / 096 | loss: 0.33267 | err: 0.14286
batch 064 / 096 | loss: 0.32967 | err: 0.14258
batch 065 / 096 | loss: 0.32491 | err: 0.14038
batch 066 / 096 | loss: 0.32872 | err: 0.14015
batch 067 / 096 | loss: 0.32382 | err: 0.13806
batch 068 / 096 | loss: 0.32409 | err: 0.13787
batch 069 / 096 | loss: 0.32459 | err: 0.13768
batch 070 / 096 | loss: 0.32378 | err: 0.13750
batch 071 / 096 | loss: 0.31984 | err: 0.13556
batch 072 / 096 | loss: 0.31965 | err: 0.13542
batch 073 / 096 | loss: 0.31865 | err: 0.13527
batch 074 / 096 | loss: 0.31686 | err: 0.13514
batch 075 / 096 | loss: 0.31610 | err: 0.13500
batch 076 / 096 | loss: 0.31578 | err: 0.13487
batch 077 / 0

batch 036 / 096 | loss: 0.40548 | err: 0.16319
batch 037 / 096 | loss: 0.40270 | err: 0.16216
batch 038 / 096 | loss: 0.39899 | err: 0.16118
batch 039 / 096 | loss: 0.40440 | err: 0.16346
batch 040 / 096 | loss: 0.40470 | err: 0.16250
batch 041 / 096 | loss: 0.40411 | err: 0.16159
batch 042 / 096 | loss: 0.40988 | err: 0.16369
batch 043 / 096 | loss: 0.40154 | err: 0.15988
batch 044 / 096 | loss: 0.41061 | err: 0.16477
batch 045 / 096 | loss: 0.40767 | err: 0.16389
batch 046 / 096 | loss: 0.40972 | err: 0.16576
batch 047 / 096 | loss: 0.40627 | err: 0.16489
batch 048 / 096 | loss: 0.39983 | err: 0.16146
batch 049 / 096 | loss: 0.40019 | err: 0.16071
batch 050 / 096 | loss: 0.40116 | err: 0.16000
batch 051 / 096 | loss: 0.39946 | err: 0.15931
batch 052 / 096 | loss: 0.39564 | err: 0.15865
batch 053 / 096 | loss: 0.38897 | err: 0.15566
batch 054 / 096 | loss: 0.39500 | err: 0.15972
batch 055 / 096 | loss: 0.39268 | err: 0.15909
batch 056 / 096 | loss: 0.39038 | err: 0.15848
batch 057 / 0

batch 014 / 048 | loss: 0.51916 | err: 0.21429
batch 015 / 048 | loss: 0.50771 | err: 0.21250
batch 016 / 048 | loss: 0.50447 | err: 0.21094
batch 017 / 048 | loss: 0.50262 | err: 0.21324
batch 018 / 048 | loss: 0.51997 | err: 0.22222
batch 019 / 048 | loss: 0.52110 | err: 0.22368
batch 020 / 048 | loss: 0.50132 | err: 0.21563
batch 021 / 048 | loss: 0.50289 | err: 0.21726
batch 022 / 048 | loss: 0.48294 | err: 0.20739
batch 023 / 048 | loss: 0.46876 | err: 0.20109
batch 024 / 048 | loss: 0.46463 | err: 0.20052
batch 025 / 048 | loss: 0.45642 | err: 0.19750
batch 026 / 048 | loss: 0.45730 | err: 0.19952
batch 027 / 048 | loss: 0.45187 | err: 0.19676
batch 028 / 048 | loss: 0.44599 | err: 0.19420
batch 029 / 048 | loss: 0.44349 | err: 0.19397
batch 030 / 048 | loss: 0.44664 | err: 0.19583
batch 031 / 048 | loss: 0.43808 | err: 0.19153
batch 032 / 048 | loss: 0.42877 | err: 0.18750
batch 033 / 048 | loss: 0.42291 | err: 0.18561
batch 034 / 048 | loss: 0.41682 | err: 0.18382
batch 035 / 0

batch 039 / 048 | loss: 0.36760 | err: 0.17147
batch 040 / 048 | loss: 0.36766 | err: 0.17188
batch 041 / 048 | loss: 0.36361 | err: 0.16921
batch 042 / 048 | loss: 0.35836 | err: 0.16667
batch 043 / 048 | loss: 0.35450 | err: 0.16424
batch 044 / 048 | loss: 0.34792 | err: 0.16051
batch 045 / 048 | loss: 0.35456 | err: 0.16528
batch 046 / 048 | loss: 0.35035 | err: 0.16304
batch 047 / 048 | loss: 0.35062 | err: 0.16356
batch 048 / 048 | loss: 0.35380 | err: 0.16536
model saved!
------------- epoch 004 / 005 | time: 014 sec | loss: 0.29047 | err: 0.12109
batch 001 / 048 | loss: 0.11386 | err: 0.06250
batch 002 / 048 | loss: 0.22415 | err: 0.12500
batch 003 / 048 | loss: 0.31535 | err: 0.14583
batch 004 / 048 | loss: 0.28079 | err: 0.12500
batch 005 / 048 | loss: 0.26621 | err: 0.11250
batch 006 / 048 | loss: 0.26049 | err: 0.11458
batch 007 / 048 | loss: 0.29431 | err: 0.13393
batch 008 / 048 | loss: 0.27799 | err: 0.12500
batch 009 / 048 | loss: 0.27133 | err: 0.12500
batch 010 / 048 |

batch 010 / 024 | loss: 0.39575 | err: 0.18750
batch 011 / 024 | loss: 0.37883 | err: 0.17898
batch 012 / 024 | loss: 0.36751 | err: 0.17448
batch 013 / 024 | loss: 0.38158 | err: 0.17788
batch 014 / 024 | loss: 0.38854 | err: 0.18304
batch 015 / 024 | loss: 0.39924 | err: 0.18750
batch 016 / 024 | loss: 0.39320 | err: 0.18359
batch 017 / 024 | loss: 0.37872 | err: 0.17647
batch 018 / 024 | loss: 0.38654 | err: 0.18056
batch 019 / 024 | loss: 0.38055 | err: 0.17763
batch 020 / 024 | loss: 0.38565 | err: 0.18125
batch 021 / 024 | loss: 0.38220 | err: 0.17857
batch 022 / 024 | loss: 0.37928 | err: 0.17756
batch 023 / 024 | loss: 0.36368 | err: 0.16984
batch 024 / 024 | loss: 0.35873 | err: 0.16667
------------- epoch 005 / 005 | time: 011 sec | loss: 0.31026 | err: 0.12891
training time: 48.17788481712341 seconds
batch 001 / 012 | loss: 0.79272 | err: 0.28125
batch 002 / 012 | loss: 0.73181 | err: 0.28125
batch 003 / 012 | loss: 0.72112 | err: 0.27604
batch 004 / 012 | loss: 0.69099 | er